In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Función para cargar y preprocesar los datos
def load_and_preprocess_data(file_path):
    with open(file_path) as f:
        data = json.load(f)
    
    # Convertir los datos en un DataFrame de Pandas
    df_list = []
    for token, records in data.items():
        for record in records:
            record['Token'] = token
            df_list.append(record)
    df = pd.DataFrame(df_list)
    
    # Eliminar columnas innecesarias y manejar valores faltantes si los hay
    df.drop(columns=['date', 'Nombre', 'Clase'], inplace=True)
    df.fillna(0, inplace=True)  # Reemplazar valores faltantes con 0
    
    # Crear la variable 'price_increase'
    df['price_increase'] = np.where(df['Price'].diff() > 0, 1, 0)
    
    return df

In [3]:
# Cargar y preprocesar los datos
data_df = load_and_preprocess_data('../data/dataset.json')

In [4]:
# Definir características y variable objetivo
features = ['Price', 'Market cap', 'tvl', 'price_trend', 'market_cap_trend', 'price_volatility', 'tvl_trend']
target = 'price_increase'  # Variable objetivo: si el precio subirá o bajará

In [5]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data_df[features], data_df[target], test_size=0.2, random_state=42)

In [6]:
# Escalar características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Construir el modelo de red neuronal
model = Sequential()
model.add(Dense(64, input_dim=len(features), activation='relu'))  # Primera capa oculta
model.add(Dense(32, activation='relu'))  # Segunda capa oculta
model.add(Dense(1, activation='sigmoid'))  # Capa de salida

In [8]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Entrenar el modelo
model.fit(X_train_scaled, y_train, epochs=50, batch_size=10, validation_data=(X_test_scaled, y_test), verbose=1)

Epoch 1/50
3118/3118 [==============================] - 7s 2ms/step - loss: 0.3529 - accuracy: 0.8579 - val_loss: 0.2452 - val_accuracy: 0.8865
Epoch 2/50
3118/3118 [==============================] - 5s 2ms/step - loss: 0.2487 - accuracy: 0.8973 - val_loss: 0.2481 - val_accuracy: 0.8785
Epoch 3/50
3118/3118 [==============================] - 6s 2ms/step - loss: 0.2357 - accuracy: 0.8999 - val_loss: 0.2527 - val_accuracy: 0.8743
Epoch 4/50
3118/3118 [==============================] - 4s 1ms/step - loss: 0.2267 - accuracy: 0.8998 - val_loss: 0.2059 - val_accuracy: 0.9122
Epoch 5/50
3118/3118 [==============================] - 6s 2ms/step - loss: 0.2209 - accuracy: 0.8998 - val_loss: 0.2005 - val_accuracy: 0.9077
Epoch 6/50
3118/3118 [==============================] - 4s 1ms/step - loss: 0.2103 - accuracy: 0.9010 - val_loss: 0.2026 - val_accuracy: 0.9080
Epoch 7/50
3118/3118 [==============================] - 4s 1ms/step - loss: 0.2303 - accuracy: 0.9020 - val_loss: 0.1853 - val_accuracy:

In [10]:
# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print("Precisión del modelo de red neuronal:", accuracy)

Precisión del modelo de red neuronal: 0.9095457792282104
